Engenharia de Dados com Python


In [ ]:
# Criando Banco de Dados no RDS
import psycopg2 as pg2
from google.colab import userdata
user = userdata.get('postgreUser')
password = userdata.get('postgrePassword')
con = pg2.connect(host="database-1.c5eueywouwye.us-east-1.rds.amazonaws.com",database="postgres",
                  user="postgreUser",password="postgrePassword")
con.autocommit = True
# cursor para fazer consultas ao banco de dados
cur = con.cursor()
cur.execute('create database inventario')
con.close()

In [ ]:
# Criando Tabela
from google.colab import userdata
user = userdata.get('postgreUser')
password = userdata.get('postgrePassword')
con = pg2.connect(host="database-1.c5eueywouwye.us-east-1.rds.amazonaws.com",database="inventario",
                  user="postgreUser",password="postgrePassword")
con.autocommit = True
# cursor para fazer consultas ao banco de dados
cur = con.cursor()
cur.execute('create table arquivos (idarquivo INT, nomearquivo VARCHAR(256));')
con.close()

In [ ]:
#Instalando o boto3
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 5.8 MB/s eta 0:00:00


In [ ]:
# Ler objetos do Bucket S3
import boto3
import io
import psycopg2
from google.colab import userdata
user = userdata.get('postgreUser')
password = userdata.get('postgrePassword')

aws_id = userdata.get('aws_id')
aws_secret = userdata.get('aws_secret')

s3 = boto3.resource(
  service_name='s3',
  region_name='us-east-1',
  aws_access_key_id = aws_id,
  aws_secret_access_key = aws_secret
)

bucket = 'engdadospythonwill'
prefix = 'imagens/'

con = pg2.connect(host="database-1.c5eueywouwye.us-east-1.rds.amazonaws.com",database="inventario",
                  user="postgreUser",password="postgrePassword")
con.autocommit = True
cur = con.cursor()
id = 0

for objects_s3 in s3.Bucket(bucket).objects.filter(Prefix=prefix):
  if objects_s3.key.endswith('jpg') or objects_s3.key.endswith('JPG'):
    filename = objects_s3.key.split('/')[1]
    #print(filename)
    # insere os objetos no banco de dados
    id += 1
    cur.execute("insert into arquivos (idarquivo,nomearquivo) values (" +str(id)+ ",'" +filename + "')")

con.close()

In [ ]:
# Lendo tabela pra conferir
import psycopg2 as pg2
from google.colab import userdata
user = userdata.get('postgreUser')
password = userdata.get('postgrePassword')

con = pg2.connect(host="database-1.c5eueywouwye.us-east-1.rds.amazonaws.com",database="inventario",
                  user="postgreUser",password="postgrePassword")
con.autocommit = True
# cursor para fazer consultas ao banco de dados
cur = con.cursor()
cur.execute('select * from arquivos;')

recset = cur.fetchall()

for rec in recset:
  print(rec)

con.close()

(1, 'avenue-g9ad8b9b60_640.jpg')
(2, 'dandelion-ge4c90edd8_640.jpg')
(3, 'fantasy-g95f970a56_640.jpg')
(4, 'garden-g42e486784_640.jpg')
(5, 'pink-ge82d54651_640.jpg')
(6, 'road-g37132565b_640.jpg')
(7, 'road-g41ea28d46_640.jpg')
(8, 'sunset-gac16749a1_640.jpg')
(9, 'tree-g386d6021c_640.jpg')
(10, 'tree-gd34ff2fcb_640.jpg')
